<a href="https://colab.research.google.com/github/ohjisu320/study_AIs/blob/main/docs/quests/MLs/RentalCarOfContractType.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

## 데이터

In [1]:
import pandas as pd
df_RCOCT = pd.read_csv('RentalCarOfContractType.csv')
df_RCOCT[:5]

,id,type_of_contract,type_of_contract2,channel,datetime,Term,payment_type,product,amount,state,overdue_count,overdue,credit rating,bank,cancellation,age,Mileage
0,66758234,렌탈,Normal,서비스 방문,2019-10-20,60,CMS,K1,96900,계약확정,0,없음,9.0,새마을금고,정상,43.0,1862.0
1,66755948,렌탈,Extension_Rental,서비스 방문,2019-10-20,60,카드이체,K1,102900,계약확정,0,없음,2.0,현대카드,정상,62.0,2532.0
2,66756657,렌탈,Normal,홈쇼핑/방송,2019-10-20,60,CMS,K1,96900,계약확정,0,없음,8.0,우리은행,정상,60.0,2363.0
3,66423450,멤버십,TAS,렌탈재계약,2019-10-20,12,CMS,K1,66900,계약확정,0,없음,5.0,농협은행,정상,60.0,2449.0
4,66423204,멤버십,TAS,렌탈재계약,2019-10-20,12,CMS,K1,66900,해약확정,12,있음,8.0,농협은행,해약,51.0,1942.0


In [107]:
df_quest = df_RCOCT[['Term', 'amount', 'overdue_count', 'age']]
df_quest.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 51304 entries, 0 to 51303
Data columns (total 4 columns):
 #   Column         Non-Null Count  Dtype  
---  ------         --------------  -----  
 0   Term           51304 non-null  int64  
 1   amount         51304 non-null  int64  
 2   overdue_count  51304 non-null  int64  
 3   age            40509 non-null  float64
dtypes: float64(1), int64(3)
memory usage: 1.6 MB


In [108]:
df_quest.isnull().sum()

Term                 0
amount               0
overdue_count        0
age              10795
dtype: int64

In [109]:
df_quest.dropna(inplace=True)

<ipython-input-109-8e3386623087>:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_quest.dropna(inplace=True)


## 모델

In [110]:
target = df_quest['age']
features = df_quest[['Term', 'amount', 'overdue_count']]
target.shape, features.shape

((40509,), (40509, 3))

In [111]:
from sklearn.model_selection import train_test_split
features_train, features_test, target_train, target_test = train_test_split(features, target, test_size=0.3, random_state=2)
features_train.shape, features_test.shape, target_train.shape, target_test.shape

((28356, 3), (12153, 3), (28356,), (12153,))

In [112]:
from sklearn.linear_model import LinearRegression
model = LinearRegression()

In [113]:
model.fit(features_train, target_train)

LinearRegression()

## 평가

In [114]:
from sklearn.metrics import r2_score
target_train_predict = model.predict(features_train)
target_train.shape, target_train_predict.shape

((28356,), (28356,))

In [115]:
r2_score(target_train, target_train_predict) # 1 이하임.

5.2057096017854576e-05

In [116]:
target_test_predict = model.predict(features_test)
target_test.shape, target_test_predict.shape

((12153,), (12153,))

In [117]:
r2_score(target_test, target_test_predict) # 1 이하임.

-0.00021001332287418073

## 전처리 적용

In [118]:
df_quest_nonnull = df_RCOCT[['Term', 'amount', 'overdue_count', 'age']]
null_index_list = df_quest_nonnull.isnull().query('age==True').index
null_index_list
features_null = df_quest_nonnull.iloc[null_index_list][['Term',	'amount',	'overdue_count']]
target_null = df_quest_nonnull.iloc[null_index_list]['age']


In [119]:
target_null_predict = model.predict(features_null)
target_null_predict.shape, target_null.shape

((10795,), (10795,))

In [120]:
df_quest_nonnull.iloc[null_index_list]['age']=target_null_predict

<ipython-input-120-8bf738c1405f>:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_quest_nonnull.iloc[null_index_list]['age']=target_null_predict


In [121]:
target_null_predict

array([46.59800006, 46.54175294, 46.58460789, ..., 46.56853728,
       46.70896312, 46.56853728])

In [122]:
df_quest_nonnull.loc[null_index_list, 'age'] = target_null_predict

In [123]:
df_quest_nonnull.loc[null_index_list, 'age']

7        46.598000
19       46.541753
21       46.584608
26       46.568537
27       46.760390
           ...    
51289    46.708963
51291    46.579251
51293    46.568537
51298    46.708963
51299    46.568537
Name: age, Length: 10795, dtype: float64

In [125]:
df_RCOCT['age']=df_quest_nonnull['age']

In [127]:
df_RCOCT.isnull().sum()

id                       0
type_of_contract         4
type_of_contract2        1
channel                  0
datetime                 0
Term                     0
payment_type             0
product                  1
amount                   0
state                    0
overdue_count            0
overdue                  2
credit rating         8783
bank                  2760
cancellation            25
age                      0
Mileage              10795
dtype: int64

In [129]:
# 재사용 위해 model을 파일로 저장
import pickle # 메모리 인스턴스 클래스를 이진파일로 저장

In [131]:
with open('RentalCarOfContractType.pkl', 'wb') as pickle_file:
  pickle.dump(obj=model, file=pickle_file)

In [132]:
df_RCOCT.to_csv('RentalCarOfContractType_age_nunnull.csv', index=False, encoding='utf-8-sig')